In [1]:
import scipy.stats as st
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt



Bad key "text.kerning_factor" on line 4 in
/Users/carolinecharrier/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df = pd.read_excel('output/df3.xlsx', sheet_name='Sheet1')
del df['Unnamed: 0']
df

,country,year_survey,quantile,income,gdpppp,country_name,gini,IGEincome
0,ALB,2008,1,728.89795,7297.00000,Albanie,30.462429,0.815874
1,ALB,2008,2,916.66235,7297.00000,Albanie,30.462429,0.815874
2,ALB,2008,3,1010.91600,7297.00000,Albanie,30.462429,0.815874
3,ALB,2008,4,1086.90780,7297.00000,Albanie,30.462429,0.815874
4,ALB,2008,5,1132.69970,7297.00000,Albanie,30.462429,0.815874
...,...,...,...,...,...,...,...,...
11594,COD,2008,96,810.62330,303.19305,"Congo, République démocratique du",44.399690,0.500000
11595,COD,2008,97,911.78340,303.19305,"Congo, République démocratique du",44.399690,0.500000
11596,COD,2008,98,1057.80740,303.19305,"Congo, République démocratique du",44.399690,0.500000
11597,COD,2008,99,1286.60290,303.19305,"Congo, République démocratique du",44.399690,0.500000


### Question 1 : n réalisation de la variable ln_y_parent 

In [3]:
def generate_incomes(n, pj):
    # On génère les revenus des parents (exprimés en logs) selon une loi normale.
    # La moyenne et variance n'ont aucune incidence sur le résultat final (ie. sur le caclul de la classe de revenu)
    ln_y_parent = st.norm(0,1).rvs(size=n)
    # Génération d'une réalisation du terme d'erreur epsilon
    residues = st.norm(0,1).rvs(size=n)
    return np.exp(pj*ln_y_parent + residues), np.exp(ln_y_parent)

def quantiles(l, nb_quantiles):
    size = len(l)
    l_sorted = l.copy()
    l_sorted = l_sorted.sort_values()
    quantiles = np.round(np.arange(1, nb_quantiles+1, nb_quantiles/size) -0.5 +1./size)
    q_dict = {a:int(b) for a,b in zip(l_sorted,quantiles)}
    return pd.Series([q_dict[e] for e in l])

def compute_quantiles(y_child, y_parents, nb_quantiles):
    y_child = pd.Series(y_child)
    y_parents = pd.Series(y_parents)
    c_i_child = quantiles(y_child, nb_quantiles)
    c_i_parent = quantiles(y_parents, nb_quantiles)
    sample = pd.concat([y_child, y_parents, c_i_child, c_i_parent], axis=1)
    sample.columns = ["y_child", "y_parents", "c_i_child","c_i_parent"]
    return sample

def distribution(counts, nb_quantiles):
    distrib = []
    total = counts["counts"].sum()
    
    if total == 0 :
        return [0] * nb_quantiles
    
    for q_p in range(1, nb_quantiles+1):
        subset = counts[counts.c_i_parent == q_p]
        if len(subset):
            nb = subset["counts"].values[0]
            distrib += [nb / total]
        else:
            distrib += [0]
    return distrib   

def conditional_distributions(sample, nb_quantiles):
    counts = sample.groupby(["c_i_child","c_i_parent"]).apply(len)
    counts = counts.reset_index()
    counts.columns = ["c_i_child","c_i_parent","counts"]
    
    mat = []
    for child_quantile in np.arange(nb_quantiles)+1:
        subset = counts[counts.c_i_child == child_quantile]
        mat += [distribution(subset, nb_quantiles)]
    return np.array(mat) 

def plot_conditional_distributions(p, cd, nb_quantiles):
    plt.figure()
    plt.gcf().set_size_inches(15, 10)
    
    # La ligne suivante sert à afficher un graphique en "stack bars", sur ce modèle : https://matplotlib.org/gallery/lines_bars_and_markers/bar_stacked.html
    cumul = np.array([0] * nb_quantiles)
    
    for i, child_quantile in enumerate(cd):
        plt.bar(np.arange(nb_quantiles)+1, child_quantile, bottom=cumul, width=0.95, label = str(i+1) +"e")
        cumul = cumul + np.array(child_quantile)

    plt.axis([.5, nb_quantiles*1.3 ,0 ,1])
    plt.title("p=" + str(p))
    plt.legend()
    plt.xlabel("quantile parents")
    plt.ylabel("probabilité du quantile enfant")
    plt.show()
    
def proba_cond(c_i_parent, c_i_child, mat):
    return mat[c_i_child, c_i_parent]

In [4]:
pj = 0.9                 # coefficient d'élasticité du pays j
nb_quantiles = 100       # nombre de quantiles (nombre de classes de revenu)
n  = 1000*nb_quantiles   # taille de l'échantillon

y_child, y_parents = generate_incomes(n, pj) # question 3

In [5]:
sample = compute_quantiles(y_child, y_parents, nb_quantiles) # question 4
sample.sort_values('c_i_child',ascending = False)

,y_child,y_parents,c_i_child,c_i_parent
6406,28.764511,4.216337,100,93
905,27.655077,4.630214,100,94
43092,43.619790,5.151984,100,96
12932,39.356679,10.418993,100,100
76277,48.086252,8.159806,100,99
...,...,...,...,...
64541,0.031364,0.243354,1,8
86201,0.025526,0.744884,1,39
76567,0.011244,0.122176,1,2
15135,0.029660,0.464293,1,23


In [7]:
# question 5
cd = conditional_distributions(sample, nb_quantiles) # question 5
print(cd)
print(cd.size)

[[0.241 0.136 0.082 ... 0.    0.    0.   ]
 [0.126 0.101 0.069 ... 0.    0.    0.   ]
 [0.085 0.071 0.066 ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.061 0.078 0.093]
 [0.    0.    0.    ... 0.086 0.09  0.122]
 [0.    0.    0.    ... 0.098 0.115 0.265]]
10000


In [8]:
# exemple question 5
c_i_child = 5 
c_i_parent = 8
p = proba_cond(c_i_parent, c_i_child, cd)
print("\nP(c_i_parent = {} | c_i_child = {}, pj = {}) = {}".format(c_i_parent, c_i_child, pj, p))


P(c_i_parent = 8 | c_i_child = 5, pj = 0.9) = 0.024


In [ ]:
# question 6
plot_conditional_distributions(pj, cd, nb_quantiles)

In [16]:
df

,country,year_survey,quantile,income,gdpppp,country_name,gini,IGEincome
0,ALB,2008,1,728.89795,7297.00000,Albanie,30.462429,0.815874
1,ALB,2008,2,916.66235,7297.00000,Albanie,30.462429,0.815874
2,ALB,2008,3,1010.91600,7297.00000,Albanie,30.462429,0.815874
3,ALB,2008,4,1086.90780,7297.00000,Albanie,30.462429,0.815874
4,ALB,2008,5,1132.69970,7297.00000,Albanie,30.462429,0.815874
...,...,...,...,...,...,...,...,...
11594,COD,2008,96,810.62330,303.19305,"Congo, République démocratique du",44.399690,0.500000
11595,COD,2008,97,911.78340,303.19305,"Congo, République démocratique du",44.399690,0.500000
11596,COD,2008,98,1057.80740,303.19305,"Congo, République démocratique du",44.399690,0.500000
11597,COD,2008,99,1286.60290,303.19305,"Congo, République démocratique du",44.399690,0.500000


In [ ]:
#question 8

pj = df['IGEincome']     # coefficient d'élasticité du pays j
nb_quantiles = 100
n  = 1000*nb_quantiles  # taille de l'échantillon

y_child, y_parents = generate_incomes(n, pj) # question 3

sample = compute_quantiles(y_child, y_parents, nb_quantiles) # question 4
sample.sort_values('c_i_child',ascending = False)

cd = conditional_distributions(sample, nb_quantiles) # question 5
print(cd)
print(cd.size)

In [ ]:
#question 9
# Dans notre cas on lui donne le tableau des 100 proba pour ratios et k = 500 pour le nombre de ligne


def hamilton_allocation(ratios, k):
    frac, results = np.modf(k * ratios)
    remainder = int(k - results.sum())
    
    indices = np.argsort(frac)[::-1]
    results[indices[0:remainder]] += 1
 
    return results.astype(int)


c_child=1
p_array = cd[c_child]
nb_per_quant = hamilton_allocation(p_array,500)
nb_per_quant

In [ ]:
nb_per_quant.sum()

In [ ]:
df

# Mission 4
Pour cette mission 4, nous chercherons à expliquer le revenu des individus en fonction de plusieurs
variables explicatives : 
- le pays de l'individu
- l'indice de Gini de ce pays
- la classe de revenus des parents, etc.

## Modèle 1 : ANOVA avec variable explicative, pays de l’individu

In [11]:
import statsmodels.formula.api
import statsmodels.api

mod = statsmodels.formula.api.ols('df.income ~ df.country', data = df).fit()
table = statsmodels.api.stats.anova_lm(mod)
table

# levene([data[data['country']=='AAA'],data[data['country']=='BBB'],...)?

,df,sum_sq,mean_sq,F,PR(>F)
df.country,115.0,5.102392e+11,4.436862e+09,98.421602,0.0
Residual,11483.0,5.176556e+11,4.508017e+07,NaN,NaN


### Version 1 : ln(parents & enfants)

### Version 2 : (parents & enfants)